In [210]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import numpy as np
from datetime import datetime
from sportsreference.ncaab.boxscore import Boxscore, Boxscores
import os 
import time
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [30]:
PATH = os.getcwd()
compact = pd.read_csv(PATH + '//data//boxscores-compact-2010-2019.csv')

In [71]:
compact_boxscores = Boxscores(datetime(2010, 10, 15), datetime(2020, 1, 17)).games

In [ ]:
df_lists = []
for date in compact_boxscore:
      df = pd.DataFrame(compact_boxscores[date])
      df['date'] = date
      df.set_index('date', inplace = True)
      df_lists.append(df)
    
compact = pd.concat(df_lists, join='outer', axis=0, sort=False)
compact = compact.drop_duplicates()

In [ ]:
URIs = recent_compact['boxscore'].to_list() # URIs for retrieving detailed boxscores
detailed = pd.DataFrame() # new dataframe

checkpoint = 200 # save to csv file every 200 posts
for i, URI in enumerate(URIs):
    
    try:
        new = Boxscore(URI).dataframe # retrieved new detailed boxscore dataframe
        detailed = pd.concat([detailed, new]) # add to master df
        print('\r', f'collected {i+1} of {len(URIs)}', end="" ) # progress
    except Exception as ex:        
        print('\n', ex, '\n') # exception handling - print the exception but continue collecting data
        continue
        
    if (i % checkpoint == 0) & (i != 0):  # saving logic
        try:
            detailed.to_csv('detailed-boxscores')
            time.sleep(1)
            checkpoint = 200
        except Exception as ex:   # if save fails try again every 50 retrievals until successful
            print('\n', ex, '\n')
            checkpoint = 50
            continue

In [85]:
detailed.to_csv('detailed_boxscores_2010-to-15-1-2020.csv') # save the final list

In [100]:
# Boxscore URI is index here, reset to column to merge with compact later
detailed = detailed.reset_index().rename(columns={'index':'boxscore'})
detailed = detailed.drop_duplicates() # remove dupes

In [177]:
# create a list of exclusive columns for each dataframe
detailed_cols = list(detailed.columns)
compact_cols = list(compact.columns)
compact_cols_exclusive = ['boxscore'] + [col for col in compact_cols if col not in detailed_cols]
compact_cols_exclusive.remove('home_abbr')
compact_cols_exclusive.remove('away_abbr')
detailed_cols_exclusive = ['boxscore', 'date'] + [col for col in detailed_cols if col not in compact_cols]
detailed_cols_exclusive.remove('home_ranking')
detailed_cols_exclusive.remove('away_ranking')
detailed_cols_exclusive.remove('home_points')
detailed_cols_exclusive.remove('away_points')

In [178]:
# slice dfs for their exclusive columns
detailed_exclusive = detailed[detailed_cols_exclusive]
compact_exclusive = compact[compact_cols_exclusive]

In [179]:
combined = pd.merge(compact_exclusive, detailed_exclusive, how='inner', on='boxscore')

In [188]:
# column for home or away win
combined['home_win'] = combined['winner'].apply(lambda x: 1 if x == 'Home' else 0)
combined['away_win'] = combined['winner'].apply(lambda x: 1 if x == 'Away' else 0)
combined = combined.drop(columns='winner')

In [207]:
print(combined['home_win'].describe(), end='\n\n')
print(combined['away_win'].describe())

count    56450.000000
mean         0.666997
std          0.471292
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: home_win, dtype: float64

count    56450.000000
mean         0.333003
std          0.471292
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: away_win, dtype: float64


In [270]:
home_cols = [col for col in combined.columns if col.split('_')[0] == 'home']
away_cols = [col for col in combined.columns if col.split('_')[0] == 'away']

In [ ]:
home = combined[['boxscore', 'date']+home_cols+['pace']]
home['playing_as'] = 'HOME'
away = combined[['boxscore', 'date']+away_cols+['pace']]
away['playing_as'] = 'AWAY'

In [272]:
team_cols = ['_'.join(col.split('_')[1:]) for col in home_cols]
team_cols = ['boxscore', 'date'] + team_cols + ['pace', 'playing_as']

In [260]:
home.columns = team_cols
away.columns = team_cols

In [273]:
home.columns = team_cols
away.columns = team_cols
teams = pd.concat([home, away])
teams = teams[['boxscore', 'date', 'name', 'playing_as'] + list(teams.columns)[3:-1]]


In [276]:
teams = teams.sort_values(by=['boxscore'])
teams.to_csv('team-individual-boxscores-2010-to-01-17-2020.csv', index=False)

In [278]:
combined.to_csv('combined-boxscores-2010-to-01-17-2020.csv', index=False)

In [280]:
detailed.to_csv('detailed-boxscores-2010-to-01-17-2020.csv', index=False)
compact.to_csv('compact-boxscores-2010-to-01-17-2020.csv', index=False)

In [282]:
compact.head(1)

,away_abbr,away_name,away_rank,away_score,boxscore,date,home_abbr,home_name,home_rank,home_score,losing_abbr,losing_name,non_di,top_25,winning_abbr,winning_name
0,rhode-island,Rhode Island,NaN,75.0,2010-11-08-pittsburgh,2010-11-08,pittsburgh,Pitt,5,83.0,rhode-island,Rhode Island,False,True,pittsburgh,Pitt
